In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import glob
from skimage.transform import resize
import os
import pandas as pd

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Loading the data

In [4]:
x_train = np.load("train_data.npy")
x_test = np.load("test_data.npy")

In [11]:
y_train = pd.read_csv("train_label.csv")
y_test = pd.read_csv("test_label.csv")
print(y_train.shape)
print(y_test.shape)

(2672, 1)
(2348, 1)


In [12]:
print(x_train.shape)
print(x_test.shape)

(2672, 224, 224, 3)
(2348, 224, 224, 3)


In [13]:
y_train.iloc[1222]

Classes    Negative
Name: 1222, dtype: object

In [14]:
y_train=y_train.replace(to_replace="Negative",value=3)
y_train=y_train.replace(to_replace="Meningioma",value=1)
y_train=y_train.replace(to_replace="Glioma",value=0)
y_train=y_train.replace(to_replace="Pituitary",value=2)

In [15]:
y_train.iloc[1200]

Classes    3
Name: 1200, dtype: int64

In [16]:
y_train.head()

,Classes
0,0
1,0
2,0
3,0
4,0


In [17]:
y_train["Classes"].value_counts()

3    1032
0     791
2     489
1     360
Name: Classes, dtype: int64

In [18]:
from keras.utils import to_categorical
y_train =to_categorical(y_train, num_classes=4)
print(y_train.shape)

Using TensorFlow backend.


(2672, 4)


In [19]:
type(y_train)

numpy.ndarray

In [20]:
y_test=y_test.replace(to_replace="Negative",value=3)
y_test=y_test.replace(to_replace="Meningioma",value=1)
y_test=y_test.replace(to_replace="Glioma",value=0)
y_test=y_test.replace(to_replace="Pituitary",value=2)

In [21]:
y_test = to_categorical(y_test, num_classes=4)

In [22]:
y_test.shape

(2348, 4)

In [23]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization
import numpy as np
import math, cv2, os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Activation, Dense, Conv2D, Reshape, concatenate, BatchNormalization, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import optimizers
from IPython.display import clear_output
%matplotlib inline
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from random import randint
from sklearn.utils import shuffle
import tensorflow as tf

In [24]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow.keras.layers import Dropout

In [74]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(hp.Int('filters', min_value=16, max_value=64, step=16), 
                     kernel_size=(3,3), padding='same', input_shape=(224, 224, 3), 
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp.Float('first_drop', min_value=0.2, max_value=0.3, step=0.05)))
    
    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(Conv2D(hp.Int('conv_{i}', min_value=32, max_value=512, step=32), 
                         kernel_size=(3,3), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(hp.Float('conv_drop', min_value=0.2, max_value=0.4, step=0.05)))
    
    model.add(Conv2D(512, 
                     kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp.Float('dropout', min_value=0.25, max_value=0.5, step=0.1)))
    
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(units=256, activation='relu', 
                    kernel_initializer='he_uniform'))
    model.add(Dropout(hp.Float('final_drop', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(units=4, activation='sigmoid'))
    
    model.compile(optimizer=hp.Choice('opt', values=["Adam", "RMSprop", "AdaDelta"]),
                  loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model

In [75]:
SEED = 10
dir='logs'

In [76]:
from kerastuner.tuners import Hyperband

In [77]:
turner = RandomSearch(build_model,
                      objective="val_accuracy",
                      max_trials=3,
                      seed=SEED,
                      directory=dir,
                      executions_per_trial=3)

INFO:tensorflow:Reloading Oracle from existing project logs\untitled_project\oracle.json


In [78]:
turner.search_space_summary()

In [ ]:
turner.search(x_train, y_train,
             epochs=50,
             batch_size=24,
             validation_data=(x_test, y_test))

In [84]:
turner.results_summary()

In [85]:
model = turner.get_best_models()[0]

In [87]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 96)      27744     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 96)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 56, 56, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 96)        8

In [88]:
model.save("cnn_model.h5")

In [90]:
turner.get_best_hyperparameters()[0].values

{'filters': 32,
 'first_drop': 0.25,
 'num_layers': 4,
 'conv_{i}': 96,
 'conv_drop': 0.39999999999999997,
 'dropout': 0.44999999999999996,
 'final_drop': 0.4000000000000001,
 'opt': 'RMSprop'}

In [91]:
hps = turner.oracle.get_best_trials(num_trials=1)[0].hyperparameters

In [92]:
model = build_model(hps)

In [93]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 112, 112, 96)      27744     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 56, 56, 96)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 56, 56, 96)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 56, 56, 96)       